## Other Collections

- So far we've been hyper-focused on `List()`

- There are other collection objects in Scala that are closer in implementation to `list` in Python
  - `Vector()` --> Faster random access, but slower prepend than list
    - All operations are the same, with the exception of `::`, which is replaced by `x +: l` for prepend, and `l :+ x` for append

- looking at the class inheritance for `Iterable` objects in Scala:
  - Iterable
    - Seq
      - List
      - Vector
      - Range
      - Array (*from Java)
    - Set
    - Map

- Instead of the single `list` object you have in Python, you have 2 in Scala
  - `List` --> Linked list for fast prepend
  - `Vector` --> For fast access
  - `Range` --> Syntactic sugar to define specific array of values 

- Also useful to note that `Array` isn't a direct descendent of `Seq` because it is a Java class, but it supports the same operations by and large

### Using `Range`

- This has kind of the same idea as python's `range`; i.e. `for i in range(20): ...`

- But it is a little more expressive. You can do
  - `1 to 5` --> [1,5]
  - `1 until 5` --> [1,5)
  - `1 to 5 by 2` --> [1,3,5]

### All operations of `Seq`

- Let s be some `Seq[T]` object of type 

- Then the following operations are valid:
  - `s.exists(p)` -> p is of type `T => Boolean`. Returns `true` if at least one `p(x)` is `true` else `false`
  - `s.forall(p)` -> p is of type `T => Boolean`. Returns `true` if all `p(x)` is `true` else `false`
  - `s.zip(s2)` -> return `Seq[(T,T)]` of pairs from `s` and `s2`
  - `s.unzip` -> The inverse of `s.zip(s2)`, will return 2 sequences where the first is element 0 of every pair, and the second is element 1 of every pair
  - `s.flatMap(f)` -> Applies `f` to all elements of `s`
  - `s.sum` -> s1 + s2 + ...
  - `s.product` -> s1 * s2 * ...
  - `s.max` -> max value
  - `s.min` -> min value

### Some illustrative applications

- See `1-slides.scala`

## `for` expressions

- Like Python, Scala also has a `for` loop syntax. This is best illustrated with an example

- Given a positive integer `n`, find all pairs of positive integers `i` and `j`, with `1 <= j < i < n` such that `i + j` is prime

- Let's first get every combination of pairs, then check the primality of their sum. 

- We could implement this in the way we did this previously, using 2 `Range` objects
  ```scala
    def getAllPairs(n: Int): Seq[(Int, Int)] = {
      (1 until n).flatMap(i =>
        (1 until i).map(j =>
          (i, j)
        )
      )
    }
  ```

- Next, assuming we have `allPairs: Seq[(Int, Int)]`, we filter out the pairs that add to a prime. For simplicity, let's reuse the `isPrime` function we wrote in `1-slides.scala`
  ```scala
    def isPrime(x: Int): Boolean = {
      val upper = Math.floor(Math.sqrt(x)).toInt
      (2 to upper).forall(v => ((x % v)!= 0))
    }

    allPairs.filter((x,y) => isPrime(x + y))
  ```

- This is not ideal because the work is spread over a few functions. But we can actually apply a more straightforward `for` syntax!
  ```scala
    for 
      i <- 1 until n
      j <- 1 until i
      if isPrime(i+j)
    yield (i,j)
  ```

### Exercise

- Write a version of scalarProduct (see last session) that makes use of a `for`:
  ```scala
    def scalarProduct(xs: List[Double], ys: List[Double]) : Double = ???
  ```

- What will the following produce?
  ```scala
    (for x <- xs; y <- ys yield x * y).sum
  ```

- See `2-slides.scala`

## Sets

- Sets are another subclass of `Seq`

- Similar to Python, `Set` holds distinct values, and are unordered

- The underlying structure is a `Hashset`, so `Set` is mainly used when there is a need to do `.contains()` operation in `O(1)`

- We'll see how to apply `Set` in the following examples

### Example: N-Queens

- This is Leetcode 51 problem

- See `3-slides.scala` for implementation

- See your Leetcode 51 solution for full explanation

## Maps and list grouping

### Maps

- `Map` is the Scala implementation of hashmaps

- `Map` is a subclass of `Iterable`, and a map is basically some collection of tuples `Iterable[(String, Int)]`

- This makes it possible to map function over Hashmaps (similar to python's dict.items()) in this way:
  
  ```scala
    val hm = Map("a" -> 1, "b" -> 2, "c" -> 3)
    hm.map((k,v) => (v, k)) //inverting the hashmap
  ```

- Similar to Python, there are 2 main ways to retrieve a value from a hashmap; via bracket notation or using "get"
  ```scala
    val hm = Map("a" -> 1, "b" -> 2, "c" -> 3)

    hm("a") //1
    hm.get("a") //1
  ```

- One gotcha here
  - Similar to Python, if you use brackets to retrieve a key and it doesn't exist, you get an error. 
  - Again, similar to python, if you use `get` to retrieve a key and it doesn't exist, you get `None`
  - In Scala, you cannot specify a custom return value for `get`. So `get` ALWAYS returns None if the key doesn't exist
  - If you want to specify a custom return value, you can use the following syntax 

  ```scala
    val hm = Map(1 -> 100, 2 -> 200).withDefault(k => k*100)

    hm(3) //300
    hm.get(3) //None
  ```

- Another important difference between bracket notation and the `get` function is: if the key exists, the bracket notation's return type follows the key's type. However, `get` will return a `Some(ValueType)` type, where `ValueType` is the type of the value in the hashmap

- This is important, because there will be times when overriding the default return behaviour becomes super important. You can do this via the usual `match...case...` syntax, where the return type is of the superclass of `Some` and `None` called **`Option`**

- To update a map, you can use either `+` or `++`

```scala
  val m = Map(1 -> 100, 2 -> 200)

  m + (3 -> 300) //add a new value to map
  m + (2 -> 300) //override existing value in map

  m ++ Map(3 -> 300, 2 -> 400) //combine 2 maps
```

### Sorting and Grouping Lists

- In Scala, `List` is kinda like a DataFrame, in the sense that you can sort and/or group a `List`

- Sorting is self explanatory; but grouping just means you create a `Map` from the `List` such that every key is a result of a user defined function `f()`, and the values for that key are the values in the original List that share the same result of `f()`

  ```scala
    val l = List(10,1,2,3,4,5)
    
    l.sorted //List(1,2,3,4,5,10)
    l.sortWith(_ < _)

    l.groupBy(x => x % 2)
  ```

### Exercise

- To apply these concepts, we'll define a class `Polynomial` that lets us to polynomial addition easily

- Start by observing that any arbitrary polynomial can be represented as a `Map`, where the keys are the polynomial degrees, and values are the polynomial coefficients

- Concept: Note that when inputs are variable length, you can use `*` to represent them
  ```scala
    def Polynom(bindings: (Int, Double)*) = ???
  ```

- What if you have multiple variable length inputs? There are 2 solutions
  ```scala
    // Use collection of Seq as input
    def Polynom(bindings1: Seq[(Int, Double)], bindings2: Seq[(Int, Double)]) = ???

    // Use separate parameter lists
    def Polynom(bindings: (Int, Double)*)(bindings2: (Int, Double)*) = ???
  ```

- See `4-slides.scala` for implementation

### Exercise 2

- Apply the `+` function using `foldLeft` 

- Interestingly, the foldLeft can be considered faster, because ++ requires you to build up an intermediate map, while foldLeft modifies in place

## Mini Project: Implement String Encoding

- In the old timey phones, you have letters associated with numbers (i.e. 2 -> "ABC", 3 -> "DEF")

- In this project, you want to create a way to encode a fixed bunch of strings into a Map

- Then, when given some sequence of numbers, you want to return all possible strings that can be generated from that number sequence
  - For example, we encode "scala" as "82252"
  - then when given 82252, return "scala" as a possible string candidate



- See `5-slides.scala`

## Assignment

- Objective: To produce `sentenceAnagrams` where given `List[String]`, produce another `List[String]` where every `String` is meaningful (i.e. is an actual word), and the characters match the input sentence exactly

- Predefined Approach
  - 3 data types
    - Word = String
    - Sentence = List[Word]
    - Occurrences = List[(Char, Int)]
  - Preloaded object
    - dictionary: List[Word] 
  - Steps
    1. Given a word compute the character occurences of the word. Using this, given a sentence, define the character occurences of the sentence
      
    2. Group the `Occurences` from `sentenceOccurences` 
    
    3. Compute sentence anagrams. Two sentences are anagrams if their `sentOccs` are the same. Don't need to worry about ordering because `sentOccs` is returned as a sorted List. Basically, we need two functions. 
    
      - One to create `dictionaryByOccurences` which takes every word in the dictionary, calls `wordOccurences`, and stores the `Occurences`. Then, do a groupBy to group all words with the same `Occurences` value together      

      - The other will create `wordAnagrams` function, which will look up `dictionaryByOccurences` to get the anagram of a given word (using occurences as the key)

    4. Compute subsets of `Occurrences`. That is, given `Occurrences = List(("a", 2), ("b", 2))`, you want to generate all possible combinations of the given character sets as follows:
    
    5. Implement `subtract` to take away counts between 2 occurences. So List(('a',2), ('b',2)) - List(('a',1)) = List(('a',1),('b',2))

    6. Putting these together, we can recursively find all anagrams of a sentence
      - Convert sentence to Occurences
      - Suppose we have `val test = List(('x', x), ('y', y), ...)`
      - Decompose into head :: tail
      - ('x', x) can take on any value from ('x', 1) to ('x', x). Call this `head`
      - Take `subtract(test, head)`
      - Then recursively call anagram on the subtracted `Occurrences`
